<a href="https://colab.research.google.com/github/jmf3192/UOC_PRACT1/blob/main/PRAC1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Comenzamos importando las librerias necesarias

import requests
import json
import pandas as pd
from bs4 import BeautifulSoup
import time
import random


# Comenzamos creando una función inicializadora que nos ayudará a identificar la
# página escecífica donde queremos hacer el scrapping.
class IMDbScraper:
    def __init__(self):
        self.url = "https://www.imdb.com/chart/top/?ref_=nv_mv_250"
        self.data = []
        self.headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36"
        }
# La siguiente función será específica para verificar el hearder y el user-agent
    def check_user_agent(self):
        print(f"User-Agent utilizado: {self.headers['User-Agent']}")

# Continuaremos creando otra funciòn que nos permita descargar la web objetivo en html
    def download_html(self):
        response = requests.get(self.url, headers=self.headers)
        if response.status_code == 200:
            return response.text
        else:
            print(f"Error al descargar la página. Estado: {response.status_code}")
            return None

# Ahora configuramos función donde podamos obtener los datos básicos de las 250 películas
    def scrape_data(self, html):
        start = html.find('<script type="application/ld+json">')
        end = html.find('</script>', start)
        json_data = html[start + len('<script type="application/ld+json">'):end].strip()

        data = json.loads(json_data)
# Para ejecutar esta función, hacemos una iteración para cada uno de los datos que queremos obtener.
        for item in data['itemListElement']:
            movie = item['item']
            title = movie['name']
            rating = movie['aggregateRating']['ratingValue']
            genre = movie.get('genre', 'N/A')
            web = movie['url']
            self.data.append([title, rating, genre, web])

# Para los datos más complejos, creamos una función a parte con BeautifulSoup.
    def scrape_movie_details(self, url):
        response = requests.get(url, headers=self.headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')

            json_data = soup.find("script", type="application/ld+json").string
            movie_data = json.loads(json_data)

            date = movie_data.get("datePublished", "N/A")
            director = ", ".join([d["name"] for d in movie_data.get("director", [])])
            writers = ", ".join([w["name"] for w in movie_data.get("creator", []) if w["@type"] == "Person"])
            actors = ", ".join([a["name"] for a in movie_data.get("actor", [])])

            time.sleep(random.uniform(1, 2))

            return date, director, writers, actors
        else:
            print(f"Error al acceder a la URL: {url}")
            return "N/A", "N/A", "N/A", "N/A"

# Ahora tenemos una función que genera el dataframe con toda la información obtenida previamente.
    def save_to_dataframe(self):
        df = pd.DataFrame(self.data, columns=["Title", "Rating", "Genre", "Web"])
        df[['Date', 'Director', 'Writers', 'Actors']] = df['Web'].apply(lambda url: pd.Series(self.scrape_movie_details(url)))

        return df


# Por último,creamos una función que ejecuta toddo el código
    def run(self):
        print("Scraping las 250 mejores películas de IMBb")

        self.check_user_agent()

        html = self.download_html()
        if html:
            self.scrape_data(html)
            df = self.save_to_dataframe()
            display(df)
            df.to_csv('films.csv', index=False)
        else:
            print("No se pudo obtener HTML.")

scraper = IMDbScraper()
scraper.run()


Scraping las 250 mejores películas de IMBb
User-Agent utilizado: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36


,Title,Rating,Genre,Web,Date,Director,Writers,Actors
0,The Shawshank Redemption,9.3,Drama,https://www.imdb.com/title/tt0111161/,1994-10-14,Frank Darabont,"Stephen King, Frank Darabont","Tim Robbins, Morgan Freeman, Bob Gunton"
1,The Godfather,9.2,"Crime, Drama",https://www.imdb.com/title/tt0068646/,1972-03-24,Francis Ford Coppola,"Mario Puzo, Francis Ford Coppola","Marlon Brando, Al Pacino, James Caan"
2,The Dark Knight,9.0,"Action, Crime, Drama",https://www.imdb.com/title/tt0468569/,2008-07-18,Christopher Nolan,"Jonathan Nolan, Christopher Nolan, David S. Goyer","Christian Bale, Heath Ledger, Aaron Eckhart"
3,The Godfather Part II,9.0,"Crime, Drama",https://www.imdb.com/title/tt0071562/,1974-12-18,Francis Ford Coppola,"Francis Ford Coppola, Mario Puzo","Al Pacino, Robert De Niro, Robert Duvall"
4,12 Angry Men,9.0,"Crime, Drama",https://www.imdb.com/title/tt0050083/,1957-04-10,Sidney Lumet,Reginald Rose,"Henry Fonda, Lee J. Cobb, Martin Balsam"
...,...,...,...,...,...,...,...,...
245,Rebecca,8.1,"Drama, Mystery, Romance",https://www.imdb.com/title/tt0032976/,1940-04-12,Alfred Hitchcock,"Daphne Du Maurier, Robert E. Sherwood, Joan Ha...","Laurence Olivier, Joan Fontaine, George Sanders"
246,The Help,8.1,Drama,https://www.imdb.com/title/tt1454029/,2011-08-10,Tate Taylor,"Tate Taylor, Kathryn Stockett","Viola Davis, Emma Stone, Octavia Spencer"
247,Cool Hand Luke,8.1,"Crime, Drama",https://www.imdb.com/title/tt0061512/,1967-11-01,Stuart Rosenberg,"Donn Pearce, Frank Pierson, Hal Dresner","Paul Newman, George Kennedy, Strother Martin"
248,"Paris, Texas",8.1,Drama,https://www.imdb.com/title/tt0087884/,1984-08-23,Wim Wenders,"L.M. Kit Carson, Sam Shepard, Walter Donohue","Harry Dean Stanton, Nastassja Kinski, Dean Sto..."


from matplotlib import pyplot as plt
_df_0['Rating'].plot(kind='hist', bins=20, title='Rating')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['Date']
  ys = series['Rating']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_1.sort_values('Date', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Date')
_ = plt.ylabel('Rating')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['Date']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'Date'}, axis=1)
              .sort_values('Date', ascending=True))
  xs = counted['Date']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_2.sort_values('Date', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Date')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
_df_3['Rating'].plot(kind='line', figsize=(8, 4), title='Rating')
plt.gca().spines[['top', 'right']].set_visible(False)